In [1]:
import os
import sys
sys.path.append("/Users/sametkoleoglu/ArtificalIntelligence/DeepLearnig/Kidney-Disease-Classification")

import zipfile
import gdown

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from src.cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.cnnClassifier.utils.common import read_yaml, create_directories
from src.cnnClassifier import logger

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([Path(self.config.artifacts_root)])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )

        logger.info(f"Data Ingestion Config: {data_ingestion_config}")
        return data_ingestion_config

In [5]:
from src.cnnClassifier.utils.common import get_size

In [6]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self) -> str:
        """
        Fetch the data from the source URL to the local directory
        """
        try:
            dataset_url = self.config.source_URL
            zip_download_path = self.config.local_data_file

            os.makedirs(os.path.dirname(zip_download_path), exist_ok=True)

            if not os.path.exists(zip_download_path):
                logger.info(f"Downloading file from :[{dataset_url}] to :[{zip_download_path}]")
                gdown.download(dataset_url, zip_download_path, quiet=False)
                logger.info("Download completed!")
            else:
                logger.info("File already exists. Skipping download.")

        except Exception as e:
            raise e
        
    def extract_zip_file(self) -> None:
        """
        Unzip the downloaded zip file into the unzip directory
        """
        try:
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)

            if not os.path.exists(self.config.local_data_file):
                raise FileNotFoundError(f"ZIP file not found: {self.config.local_data_file}")

            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            
            logger.info(f"Extracted zip file to :[{unzip_path}]")

        except Exception as e:
            raise e


In [7]:
try:
    config_manager = ConfigurationManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-11-30 10:02:30,201: INFO: common]: yaml file: ../config/config.yaml loaded successfully
[2025-11-30 10:02:30,202: INFO: common]: yaml file: ../params.yaml loaded successfully
[2025-11-30 10:02:30,203: INFO: common]: created directory at: artifacts
[2025-11-30 10:02:30,203: INFO: 329316465]: Data Ingestion Config: DataIngestionConfig(root_dir='artifacts/data_ingestion', source_URL='https://drive.google.com/uc?id=1ii_S5SGFQnTOnaXv5ApIY0X9BtE0RLcs', local_data_file='artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone.zip', unzip_dir='artifacts/data_ingestion')
[2025-11-30 10:02:30,203: INFO: 396452231]: Downloading file from :[https://drive.google.com/uc?id=1ii_S5SGFQnTOnaXv5ApIY0X9BtE0RLcs] to :[artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone.zip]


Downloading...
From (original): https://drive.google.com/uc?id=1ii_S5SGFQnTOnaXv5ApIY0X9BtE0RLcs
From (redirected): https://drive.google.com/uc?id=1ii_S5SGFQnTOnaXv5ApIY0X9BtE0RLcs&confirm=t&uuid=b0987764-993d-48d3-be0f-1e77329220b3
To: /Users/sametkoleoglu/ArtificalIntelligence/DeepLearnig/Kidney-Disease-Classification/research/artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone.zip
100%|██████████| 1.63G/1.63G [01:58<00:00, 13.8MB/s]

[2025-11-30 10:04:31,008: INFO: 396452231]: Download completed!


[2025-11-30 10:04:42,626: INFO: 396452231]: Extracted zip file to :[artifacts/data_ingestion]
